# Personal Knowledge Worker Using RAG

Tool for querying personal file storage using RAG. Working with local Llama instance

In [ ]:
! pip -q install langchain langchain-community sentence-transformers ollama

In [ ]:
# imports

import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import glob
from langchain_community.llms import Ollama
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import gradio as gr
from langchain_core.callbacks import StdOutCallbackHandler


In [ ]:
# Using mistral model to run locally

MODEL = "mistral"
db_name = "vector_db"

In [ ]:
def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc


# Using reference data as default to avoid repeating the same steps. This can be substituted with any other data later dynamically
def load_documents(folders = glob.glob("../../knowledge-base/*")):
    text_loader_kwargs = {'encoding': 'utf-8'}

    documents = []
    for folder in folders:
        doc_type = os.path.basename(folder)
        loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
        folder_docs = loader.load()
        documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)

    print(f"Total number of chunks: {len(chunks)}")
    print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

    return documents, chunks


In [ ]:
embeddings = OllamaEmbeddings(model=MODEL)

documents, chunks = load_documents()

# Use existing vectorstore if it exists
if os.path.exists(db_name):
    vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
else:
    # Create a new vectorstore
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
llm = Ollama(model=MODEL)

In [ ]:
retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
question = "Please explain what Insurellm is in a couple of sentences"
result = conversation_chain.invoke(question)

In [ ]:
print(result["answer"])

In [ ]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)